In [1]:
import pandas as pd
import numpy as np
import sys
import time
import xgboost as xgb
from sklearn.externals.joblib import dump, load, Parallel, delayed

In [2]:
import utils
from utils import *

In [6]:
t0org0 = pd.read_csv("train/train.csv")

In [7]:
import scipy as sc
import scipy.sparse as sp
from sklearn.utils import check_random_state  
import sys
import time

In [11]:
t0org=t0org0

In [12]:
t0org['day']=np.round(t0org.hour % 10000 / 100)#日期
t0org['hour1'] = np.round(t0org.hour % 100)#时间
t0org['day_hour'] = (t0org.day.values - 21) * 24 + t0org.hour1.values#距离21日的小时数
t0org['day_hour_prev'] = t0org['day_hour'] - 1#前一小时
t0org['day_hour_next'] = t0org['day_hour'] + 1#后一小时
t0org['app_or_web'] = 0
t0org.ix[t0org.app_id.values=='ecad2386', 'app_or_web'] = 1
t0 = t0org
t0['app_site_id'] = np.add(t0.app_id.values, t0.site_id.values)
calc_exptv(t0,  ['app_or_web'])

exptv_vn_list = ['app_site_id', 'as_domain', 'C14','C17', 'C21', 'device_model', 'device_ip', 'device_id', 'dev_ip_aw', 
                'app_site_model', 'site_model','app_model', 'dev_id_ip', 'C14_aw', 'C17_aw', 'C21_aw']

calc_exptv(t0, exptv_vn_list)

calc_exptv(t0, ['app_site_id'], add_count=True)
vns = ['app_or_web',  'device_ip', 'app_site_id', 'device_model', 'app_site_model', 'C1', 'C14', 'C17', 'C21',
                        'device_type', 'device_conn_type','app_site_model_aw', 'dev_ip_app_site']
dftv = t0.ix[np.logical_and(t0.day.values >= 21, t0.day.values < 32), ['click', 'day', 'id'] + vns].copy()

dftv['app_site_model'] = np.add(dftv.device_model.values, dftv.app_site_id.values)
dftv['app_site_model_aw'] = np.add(dftv.app_site_model.values, dftv.app_or_web.astype('string').values)
dftv['dev_ip_app_site'] = np.add(dftv.device_ip.values, dftv.app_site_id.values)
for vn in vns:
    dftv[vn] = dftv[vn].astype('category')
    print(vn)

n_ks = {'app_or_web': 100, 'app_site_id': 100, 'device_ip': 10, 'C14': 50, 'app_site_model': 50, 'device_model': 100, 'device_id': 50,
        'C17': 100, 'C21': 100, 'C1': 100, 'device_type': 100, 'device_conn_type': 100, 'banner_pos': 100,
        'app_site_model_aw': 100, 'dev_ip_app_site': 10 , 'device_model': 500}

exp2_dict = {}
for vn in vns:
    exp2_dict[vn] = np.zeros(dftv.shape[0])

days_npa = dftv.day.values
    
for day_v in xrange(22, 32):
    df1 = dftv.ix[np.logical_and(dftv.day.values < day_v, dftv.day.values < 31), :].copy()
    df2 = dftv.ix[dftv.day.values == day_v, :]
    print("Validation day:", day_v, ", train data shape:", df1.shape, ", validation data shape:", df2.shape)
    pred_prev = df1.click.values.mean() * np.ones(df1.shape[0])
    for vn in vns:
        if 'exp2_'+vn in df1.columns:
            df1.drop('exp2_'+vn, inplace=True, axis=1)
    for i in range(3):
        for vn in vns:
            p1 = calcLeaveOneOut2(df1, vn, 'click', n_ks[vn], 0, 0.25, mean0=pred_prev)
            pred = pred_prev * p1
            print(day_v, i, vn, "change = ", ((pred - pred_prev)**2).mean())
            pred_prev = pred    
            
        pred1 = df1.click.values.mean()
        for vn in vns:
            print("="*20, "merge", day_v, vn)
            diff1 = mergeLeaveOneOut2(df1, df2, vn)
            pred1 *= diff1
            exp2_dict[vn][days_npa == day_v] = diff1
        
        pred1 *= df1.click.values.mean() / pred1.mean()
        print("logloss = ", logloss(pred1, df2.click.values))
        #print my_lift(pred1, None, df2.click.values, None, 20, fig_size=(10, 5))
        #plt.show()

for vn in vns:
    t0['exp2_'+vn] = exp2_dict[vn]


print("to count prev/current/next hour by ip ...")
cntDualKey(t0, 'device_ip', None, 'day_hour', 'day_hour_prev', fill_na=0)
cntDualKey(t0, 'device_ip', None, 'day_hour', 'day_hour', fill_na=0)
cntDualKey(t0, 'device_ip', None, 'day_hour', 'day_hour_next', fill_na=0)

print("to create day diffs")
t0['pday'] = t0.day - 1
calcDualKey(t0, 'device_ip', None, 'day', 'pday', 'click', 10, None, True, True)
t0['cnt_diff_device_ip_day_pday'] = t0.cnt_device_ip_day.values  - t0.cnt_device_ip_pday.values
t0['hour1_web'] = t0.hour1.values
t0.ix[t0.app_or_web.values==0, 'hour1_web'] = -1
t0['app_cnt_by_dev_ip'] = my_grp_cnt(t0.device_ip.values.astype('string'), t0.app_id.values.astype('string'))


t0['hour1'] = np.round(t0.hour.values % 100)
t0['cnt_diff_device_ip_day_pday'] = t0.cnt_device_ip_day.values  - t0.cnt_device_ip_pday.values

t0['rank_dev_ip'] = my_grp_idx(t0.device_ip.values.astype('string'), t0.id.values.astype('string'))
t0['rank_day_dev_ip'] = my_grp_idx(np.add(t0.device_ip.values, t0.day.astype('string').values).astype('string'), t0.id.values.astype('string'))
t0['rank_app_dev_ip'] = my_grp_idx(np.add(t0.device_ip.values, t0.app_id.values).astype('string'), t0.id.values.astype('string'))


t0['cnt_dev_ip'] = get_agg(t0.device_ip.values, t0.id, np.size)
t0['cnt_dev_id'] = get_agg(t0.device_id.values, t0.id, np.size)

t0['dev_id_cnt2'] = np.minimum(t0.cnt_dev_id.astype('int32').values, 300)
t0['dev_ip_cnt2'] = np.minimum(t0.cnt_dev_ip.astype('int32').values, 300)

t0['dev_id2plus'] = t0.device_id.values
t0.ix[t0.cnt_dev_id.values == 1, 'dev_id2plus'] = '___only1'
t0['dev_ip2plus'] = t0.device_ip.values
t0.ix[t0.cnt_dev_ip.values == 1, 'dev_ip2plus'] = '___only1'

t0['diff_cnt_dev_ip_hour_phour_aw2_prev'] = (t0.cnt_device_ip_day_hour.values - t0.cnt_device_ip_day_hour_prev.values) * ((t0.app_or_web * 2 - 1)) 
t0['diff_cnt_dev_ip_hour_phour_aw2_next'] = (t0.cnt_device_ip_day_hour.values - t0.cnt_device_ip_day_hour_next.values) * ((t0.app_or_web * 2 - 1)) 


print("to save t0 ...")

dump(t0, tmp_data_path + 't0.joblib_dat')


print("to generate t0tv_mx .. ")
app_or_web = None
_start_day = 22
list_param = ['C1', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'banner_pos', 'device_type', 'device_conn_type']
feature_list_dict = {}

feature_list_name = 'tvexp3'
feature_list_dict[feature_list_name] = list_param + \
                            ['exptv_' + vn for vn in ['app_site_id', 'as_domain', 
                             'C14','C17', 'C21', 'device_model', 'device_ip', 'device_id', 'dev_ip_aw', 
                             'dev_id_ip', 'C14_aw', 'C17_aw', 'C21_aw']] + \
                            ['cnt_diff_device_ip_day_pday', 
                             'app_cnt_by_dev_ip', 'cnt_device_ip_day_hour', 'app_or_web',
                             'rank_dev_ip', 'rank_day_dev_ip', 'rank_app_dev_ip',
                             'diff_cnt_dev_ip_hour_phour_aw2_prev', 'diff_cnt_dev_ip_hour_phour_aw2_next',
                             'exp2_device_ip', 'exp2_app_site_id', 'exp2_device_model', 'exp2_app_site_model',
                             'exp2_app_site_model_aw', 'exp2_dev_ip_app_site',
                             'cnt_dev_ip', 'cnt_dev_id', 'hour1_web']

filter_tv = np.logical_and(t0.day.values >= _start_day, t0.day.values < 31)
filter_t1 = np.logical_and(t0.day.values < 30, filter_tv)
filter_v1 = np.logical_and(~filter_t1, filter_tv)    
    
print(filter_tv.sum())


for vn in feature_list_dict[feature_list_name] :
    if vn not in t0.columns:
        print("="*60 + vn)
        
yv = t0.click.values[filter_v1]

t0tv_mx = t0.as_matrix(feature_list_dict[feature_list_name])

print(t0tv_mx.shape)

C:\Users\shachuanyu\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


mean0: 0.17419812539185714
app_or_web app_or_web   22  done in  26.279731273651123
mean0: 0.16458964954042343
app_or_web app_or_web   23  done in  14.735992908477783
mean0: 0.16969998889045002
app_or_web app_or_web   24  done in  12.127697229385376
mean0: 0.17070377576138496
app_or_web app_or_web   25  done in  12.771343469619751
mean0: 0.17266801725492412
app_or_web app_or_web   26  done in  17.210146188735962
mean0: 0.17433656192708133
app_or_web app_or_web   27  done in  13.378575801849365
mean0: 0.17520055869652343
app_or_web app_or_web   28  done in  29.518617391586304
mean0: 0.17144169697765613
app_or_web app_or_web   29  done in  19.11216115951538
mean0: 0.1698667791732506
app_or_web app_or_web   30  done in  22.935389518737793
mean0: 0.16980562476404604
app_or_web app_or_web   31  done in  7.338051080703735
app_or_web app_or_web 22 5337126 5337126
app_or_web app_or_web 23 3870752 3870752
app_or_web app_or_web 24 3335302 3335302
app_or_web app_or_web 25 3363122 3363122
app_or_we

IndexError: boolean index did not match indexed array along dimension 0; dimension is 40428967 but corresponding boolean dimension is 9460121

In [13]:
t0.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,C20,C21,day,hour1,day_hour,day_hour_prev,day_hour_next,app_or_web,app_site_id,exptv_app_or_web
0,1.000009e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,-1,79,21.0,0,0.0,-1.0,1.0,1,ecad23861fbe01fe,0.0
1,1.000017e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,100084,79,21.0,0,0.0,-1.0,1.0,1,ecad23861fbe01fe,0.0
2,1.000037e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,100084,79,21.0,0,0.0,-1.0,1.0,1,ecad23861fbe01fe,0.0
3,1.000064e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,100084,79,21.0,0,0.0,-1.0,1.0,1,ecad23861fbe01fe,0.0
4,1.000068e+19,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,...,-1,157,21.0,0,0.0,-1.0,1.0,1,ecad2386fe8cc448,0.0


# lets go

In [2]:
import pandas as pd 
import numpy as np

In [3]:
import utils
from utils import *

In [4]:
reader = pd.read_csv("train/train.csv",iterator=True)
t0org= reader.get_chunk(500000)

In [5]:
t0org['day']=np.round(t0org.hour % 10000 / 100)
t0org['hour1'] = np.round(t0org.hour % 100)
t0org['day_hour'] = (t0org.day.values - 21) * 24 + t0org.hour1.values
t0org['day_hour_prev'] = t0org['day_hour'] - 1
t0org['day_hour_next'] = t0org['day_hour'] + 1
t0org['app_or_web'] = 0
t0org.ix[t0org.app_id.values=='ecad2386', 'app_or_web'] = 1

C:\Users\shachuanyu\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


In [4]:
t0org.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,C18,C19,C20,C21,day,hour1,day_hour,day_hour_prev,day_hour_next,app_or_web
0,1.000009e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,0,35,-1,79,21.0,0,0.0,-1.0,1.0,1
1,1.000017e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,0,35,100084,79,21.0,0,0.0,-1.0,1.0,1
2,1.000037e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,0,35,100084,79,21.0,0,0.0,-1.0,1.0,1
3,1.000064e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,0,35,100084,79,21.0,0,0.0,-1.0,1.0,1
4,1.000068e+19,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,...,0,35,-1,157,21.0,0,0.0,-1.0,1.0,1


In [6]:
t0 = t0org

t0['app_site_id'] = np.add(t0.app_id.values, t0.site_id.values)

In [7]:
t0a = t0.ix[:, ['day', 'click']].copy()
day_exps = {}
t0a['app_or_web'] = t0['app_or_web']
t0a.head()

C:\Users\shachuanyu\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


,day,click,app_or_web
0,21.0,0,1
1,21.0,0,1
2,21.0,0,1
3,21.0,0,1
4,21.0,0,1


In [8]:
vn ='app_or_web'

In [9]:
cred_k = 10
day_v =22
if day_v not in day_exps:
    day_exps[day_v] = {}#day_exps[22]={}
vn_key = vn
day1 = 20
filter_t = np.logical_and(t0.day.values > day1, t0.day.values <= day_v)#day1 ->day_v
vn_key = vn
t1 = t0a.ix[filter_t, :].copy()
filter_t2 = np.logical_and(t1.day.values != day_v, t1.day.values < 31)

C:\Users\shachuanyu\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


In [10]:
df=t1
vn='app_or_web'
vn_y='click'
filter_train=filter_t2
mean0 = df.ix[filter_train, vn_y].mean()
print("mean0:", mean0)

mean0: 0.1749


C:\Users\shachuanyu\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


In [14]:
df['_key1'] = df[vn].astype('category').values.codes

In [18]:
df[vn].astype('category').values.codes

array([1, 1, 1, ..., 1, 1, 1], dtype=int8)

In [19]:
df_yt = df.ix[filter_train, ['_key1', vn_y]]

C:\Users\shachuanyu\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [21]:
grp1 = df_yt.groupby(['_key1'])

In [22]:
grp1

In [23]:
sum1 = grp1[vn_y].aggregate(np.sum)
cnt1 = grp1[vn_y].aggregate(np.size)

In [24]:
sum1

_key1
0     2331
1    15159
Name: click, dtype: int64

In [25]:
cnt1

_key1
0    21773
1    78227
Name: click, dtype: int64

In [26]:
grp1[vn_y]

In [29]:
v_codesq = df.ix[filter_train, '_key1']

C:\Users\shachuanyu\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [31]:
v_codesq

0        1
1        1
2        1
3        1
4        1
5        1
6        1
7        1
8        1
9        1
10       1
11       1
12       1
13       1
14       0
15       1
16       1
17       1
18       1
19       0
20       1
21       1
22       1
23       1
24       1
25       0
26       1
27       0
28       1
29       1
        ..
99970    0
99971    1
99972    1
99973    1
99974    1
99975    1
99976    1
99977    0
99978    1
99979    0
99980    1
99981    1
99982    1
99983    1
99984    1
99985    1
99986    1
99987    1
99988    1
99989    1
99990    1
99991    1
99992    1
99993    1
99994    1
99995    0
99996    1
99997    1
99998    1
99999    1
Name: _key1, Length: 100000, dtype: int8

In [39]:
_sum = sum1[v_codesq]

In [41]:
np.isnan(_sum).unique()

array([False])

In [45]:
_sum = sum1[v_codesq].values    
_cnt = cnt1[v_codesq].values

In [48]:
np.unique((_sum + cred_k * mean0)/(_cnt + cred_k))

array([0.10709035, 0.19377978])

In [51]:
t0

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,C20,C21,day,hour1,day_hour,day_hour_prev,day_hour_next,app_or_web,app_site_id,exptv_app_or_web
0,1.000009e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,-1,79,21.0,0,0.0,-1.0,1.0,1,ecad23861fbe01fe,0.0
1,1.000017e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,100084,79,21.0,0,0.0,-1.0,1.0,1,ecad23861fbe01fe,0.0
2,1.000037e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,100084,79,21.0,0,0.0,-1.0,1.0,1,ecad23861fbe01fe,0.0
3,1.000064e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,100084,79,21.0,0,0.0,-1.0,1.0,1,ecad23861fbe01fe,0.0
4,1.000068e+19,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,...,-1,157,21.0,0,0.0,-1.0,1.0,1,ecad2386fe8cc448,0.0
5,1.000072e+19,0,14102100,1005,0,d6137915,bb1ef334,f028772b,ecad2386,7801e8d9,...,100077,117,21.0,0,0.0,-1.0,1.0,1,ecad2386d6137915,0.0
6,1.000072e+19,0,14102100,1005,0,8fda644b,25d4cfcd,f028772b,ecad2386,7801e8d9,...,-1,157,21.0,0,0.0,-1.0,1.0,1,ecad23868fda644b,0.0
7,1.000092e+19,0,14102100,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,...,-1,23,21.0,0,0.0,-1.0,1.0,1,ecad2386e151e245,0.0
8,1.000095e+19,1,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,-1,79,21.0,0,0.0,-1.0,1.0,1,ecad23861fbe01fe,0.0
9,1.000126e+19,0,14102100,1002,0,84c7ba46,c4e18dd6,50e219e0,ecad2386,7801e8d9,...,100191,23,21.0,0,0.0,-1.0,1.0,1,ecad238684c7ba46,0.0


In [7]:
df=t0
vn='device_ip'
key_src='day_hour'
key_tgt='day_hour_prev'

In [64]:
df[key_src].astype('str').values+df[vn].astype('str').values

array(['0.0ddd2926e', '0.096809ac8', '0.0b3cf8def', ..., '0.085cd2524',
       '0.0eee01b6c', '0.0f5c6249f'], dtype=object)

In [65]:
_key_src = np.add(df[key_src].astype('str').values, df[vn].astype('str').values)
_key_tgt = np.add(df[key_tgt].astype('str').values, df[vn].astype('str').values)

In [66]:
grp1 = df.groupby(_key_src)
cnt1 = grp1[vn].aggregate(np.size)

In [70]:
cnt1

0.00000c448     1
0.00001095d     3
0.000011a60     1
0.00001e083     1
0.00002f7e4     2
0.00007786f    10
0.0000a9b24     1
0.0000b4355     7
0.0000c7862     2
0.0000d10d9     1
0.0000dcf99    19
0.0000f0192     1
0.0001131b7     1
0.00015e50c     3
0.00016cd85     1
0.00018335d     1
0.00018bcf1     1
0.0001a1436     3
0.0001b5984     1
0.000212eac     2
0.0002652ec     1
0.0002761ca     1
0.0002ac99a     2
0.0002d5c4d     1
0.0002dabe2     1
0.0002f157a     5
0.00032c849     6
0.00032e67d     1
0.000344385     1
0.000359986     1
               ..
0.0ffcc2a48     2
0.0ffced477     1
0.0ffcfd38c     1
0.0ffd63159     9
0.0ffd89488     1
0.0ffd93594     1
0.0ffd962f8     1
0.0ffd9fe1d     1
0.0ffdc1c3d     2
0.0ffde9aae     1
0.0ffdf6b89     1
0.0ffe1723c     1
0.0ffe1b044     1
0.0ffe20638     1
0.0ffe2b496     6
0.0ffe846c3     1
0.0ffec2129     2
0.0ffee7780     2
0.0fff17e31     1
0.0fff1adc2     1
0.0fff3aee7     1
0.0fff43a9d     1
0.0fff681f0     1
0.0fff9d9bd     1
0.0fffa914

In [1]:
cnt1[_key_tgt].sort()

NameError: name 'cnt1' is not defined

In [8]:
t0

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,C19,C20,C21,day,hour1,day_hour,day_hour_prev,day_hour_next,app_or_web,app_site_id
0,1.000009e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,35,-1,79,21.0,0,0.0,-1.0,1.0,1,ecad23861fbe01fe
1,1.000017e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,35,100084,79,21.0,0,0.0,-1.0,1.0,1,ecad23861fbe01fe
2,1.000037e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,35,100084,79,21.0,0,0.0,-1.0,1.0,1,ecad23861fbe01fe
3,1.000064e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,35,100084,79,21.0,0,0.0,-1.0,1.0,1,ecad23861fbe01fe
4,1.000068e+19,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,...,35,-1,157,21.0,0,0.0,-1.0,1.0,1,ecad2386fe8cc448
5,1.000072e+19,0,14102100,1005,0,d6137915,bb1ef334,f028772b,ecad2386,7801e8d9,...,431,100077,117,21.0,0,0.0,-1.0,1.0,1,ecad2386d6137915
6,1.000072e+19,0,14102100,1005,0,8fda644b,25d4cfcd,f028772b,ecad2386,7801e8d9,...,39,-1,157,21.0,0,0.0,-1.0,1.0,1,ecad23868fda644b
7,1.000092e+19,0,14102100,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,...,39,-1,23,21.0,0,0.0,-1.0,1.0,1,ecad2386e151e245
8,1.000095e+19,1,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,35,-1,79,21.0,0,0.0,-1.0,1.0,1,ecad23861fbe01fe
9,1.000126e+19,0,14102100,1002,0,84c7ba46,c4e18dd6,50e219e0,ecad2386,7801e8d9,...,167,100191,23,21.0,0,0.0,-1.0,1.0,1,ecad238684c7ba46


In [24]:
t0.iloc[499999]

id                       1.31121e+18
click                              1
hour                        14102103
C1                              1005
banner_pos                         0
site_id                     2328ee8e
site_domain                 7804dea6
site_category               f028772b
app_id                      ecad2386
app_domain                  7801e8d9
app_category                07d7df22
device_id                   a99f214a
device_ip                   f1a323ab
device_model                1f0bc64f
device_type                        1
device_conn_type                   0
C14                             6563
C15                              320
C16                               50
C17                              572
C18                                2
C19                               39
C20                               -1
C21                               32
day                               21
hour1                              3
day_hour                           3
d

In [10]:
_key_src = np.add(df[key_src].astype('str').values, df[vn].astype('str').values)
_key_tgt = np.add(df[key_tgt].astype('str').values, df[vn].astype('str').values)

In [13]:
_key_src

array(['0.0ddd2926e', '0.096809ac8', '0.0b3cf8def', ..., '3.0396a38a8',
       '3.0fa7a267e', '3.0f1a323ab'], dtype=object)

In [14]:
grp1 = df.groupby(_key_src)
cnt1 = grp1[vn].aggregate(np.size)

In [27]:
cnt1

0.00000c448     1
0.00001095d     3
0.000011a60     1
0.00001e083     1
0.00002f7e4     2
0.00007786f    10
0.0000a9b24     1
0.0000b4355    11
0.0000c7862     2
0.0000d10d9     1
0.0000dcf99    25
0.0000f0192     2
0.0000f426b     1
0.000110b71     1
0.0001131b7     1
0.00015e50c     3
0.00016cd85     1
0.00018335d     1
0.00018bcf1     1
0.0001a1436     5
0.0001b5984     1
0.000212eac     3
0.0002652ec     1
0.0002761ca     1
0.0002ac99a     2
0.0002d5c4d     1
0.0002dabe2     1
0.0002f157a     7
0.00032c849     8
0.00032e67d     1
               ..
3.0ffbd796a     7
3.0ffbe84e7     1
3.0ffc168e7     2
3.0ffc25b33     1
3.0ffc2bacf     1
3.0ffc4f40c     1
3.0ffc94f54     1
3.0ffcce76b     1
3.0ffcceaa8     1
3.0ffce7771     1
3.0ffcf52fe     1
3.0ffcfb17b     1
3.0ffd265d7     1
3.0ffd7d83d     1
3.0ffda7c89     1
3.0ffdc37d3     1
3.0ffe760c3     1
3.0ffe9bb19     1
3.0ffea2743     1
3.0ffeacecf     1
3.0ffee1a8c     1
3.0ffee2444     1
3.0fff209f8     1
3.0fff3f645     1
3.0fff41eb

In [16]:
_cnt = cnt1[_key_tgt].values

In [17]:
_cnt

array([nan, nan, nan, ..., nan, nan,  5.])

In [29]:
cnt1[_key_tgt].values

array([nan, nan, nan, ..., nan, nan,  5.])